# Autogen Ai Agents Basics

This curated notebook demonstrates autogen ai agents basics.

## Contents
1. Setup
2. Tutorial
3. Next Steps


# AI Agents Basics with AutoGen


In [ ]:
# TODO: describe cell
!pip install pyautogen python-dotenv autogen

In [ ]:
# TODO: describe cell
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# TODO: describe cell
llm_config = {"model": "gpt-4o-mini"}

## Define single Agent

In [ ]:
# TODO: describe cell
from autogen import ConversableAgent

In [ ]:
# TODO: describe cell
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

In [ ]:
# TODO: describe cell
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke", "role": "user"}]
)
print(reply)

## Conversation between two Conversable Agents

In [ ]:
# TODO: describe cell
cathy = ConversableAgent(
    name="cathy",
    system_message="Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

joe = ConversableAgent(
    name="joe",
    system_message=
      "Your name is Cathy and you are a stand-up comedian."
      "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

In [ ]:
# TODO: describe cell
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling!",
    max_turns=2,
    #is_termination_msg=lambda msg: "I gotta go" in msg["content"]
    summary_method = "reflection_with_llm",
    summary_prompt="Summarize the conversation"
)

In [ ]:
# TODO: describe cell
import pprint
pprint.pprint(chat_result.chat_history)
pprint.pprint(chat_result.cost)
pprint.pprint(chat_result.summary) #summary based on above settings

## Using of tools

In [ ]:
# TODO: describe cell
from typing import Annotated, Literal
Operator = Literal["+", "-", "*", "/"]

def calculator(a:int, b:int, operator: Annotated[Operator, "operator"]) -> int:
  """A simple calculator tool"""
  if operator == "+":
      return a + b
  elif operator == "-":
      return a - b
  else:
    raise ValueError("Invalid operator")

In [ ]:
# TODO: describe cell
math_expert_agent = ConversableAgent(
    name="MathExpert",
    system_message=
      "You are very good at math "
      "and you help users to solve thei math problems.",
    llm_config=llm_config,
    function_map={"calculator": calculator},
    human_input_mode="NEVER"
)

In [ ]:
# TODO: describe cell
math_expert_agent.register_for_llm(name="calculator", description="A tool to make math operations")(calculator)

In [ ]:
# TODO: describe cell
import json
reply = math_expert_agent.generate_reply(
    messages=[{"content":"What is 2+2?", "role" : "user"}]
)

## Build a team of Agents (sketch)

**UserProxy**: to allow the user to comment on the report and ask the writer to refine it

**Planner**: to determine relevant info needed to complete the task

**Engineer**: to write code using the defined plan by the planner

**Executor**: to execute the code written by the engineer

**Writer**: to write report

In [ ]:
# TODO: describe cell
import autogen

In [ ]:
# TODO: describe cell
user_proxy = autogen.ConversableAgent(
    name="user_proxy",
    system_message="Give the task and send instrctions to writer to refine the blog post",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

...

In [ ]:
# TODO: describe cell
grouptchat = autogen.GroupChat(
    agents=[user_proxy],
    messages=[],
    max_round=10
)

In [ ]:
#to determinate which agent should be involved
manager = autogen.GroupChatManager(
    groupchat=grouptchat,
    llm_config=llm_config,
)

In [ ]:
# TODO: describe cell
task = "<Define a task>"

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task
)

In [ ]:
# TODO: describe cell